# Cropping Plot Images to the Center 2/3rds 

This Notebook uses openCV to crop the plot images made in "Rename_Images.ipynb" and saves all dates to a new directory. It also relables the images as "Plot#_YYYYMMDD" so that the date follows the image as four-digit year and converts the file format from a jpeg to a tiff to avoid compression.

-Luke Rosen

2/16/2022

Update 3/10/2022

Creates an offset to remove the cart. 

Cuts the image to a square

4/18/2022

Makes paths relative



In [1]:
import os
import glob
import shutil
import numpy as np
import cv2

## Environment and Directories

The packages used in notebook are in the ag_env. Os, glob, and shultil are used to navigate, and copy directories and files. OpenCV and numpy are used to crop the images. 

All the files cropped are in the "images labeled" directory in the jpeg file format. Once cropped, they are written to "images_prepared" as a tif format. 

In [2]:
src_dir = "./images_labeled/"  ##Source directory
if not os.path.exists("./images_prepared"):
    os.mkdir("./images_prepared")

new_dir = "./images_prepared/" ##New directory.

## Cropping Function

The next cell creates a cropping function to crop the image to the near center based in input divide and the location of the cart (or other noninterest objects). First, the output name is created by taking the date of the folder and splitting it to year, month, day. Next, the image is open using OpenCV. The image is then cut to a square and recentered depending on if image taller than wide or vice versa. Another offset is added based on where noninterest objects (e.g. the cart bar) are in the image. These exception offsets were found experimentally by looking at images without offsets and meauring the cart bar.

Once the input  of the cropping amount is put in the fucntion (in this case, the interger 3 will divide it into thirds), the image will be cropped based onthe sum of the inputs and offsets.





In [3]:


def crop_image(jpeg= None, image=None, year="0000", month="00", day="00", divide=3, height_exception= -500, width_exception= 500):   
    if len(month) == 1:
        month = "0" + month ##Adds a leading "0" if month is single digit
    if len(day) == 1:
        day = "0" + day     ##Adds a leading "0" if day is single digit
    plot = image.split(".")[0] ##removes ".tif" from image name
    img = cv2.imread(jpeg)
    height = img.shape[0]   ##Image Height
    width = img.shape[1]    ##Image Witdth
    if height>= width:
        height_offset = int((height - width)/2) ## Centers the new height
        width_offset = 0                        ## Width center will not change
        height = width                          ## Turns area into a square
    else:
        height_offset = 0                       ## Height center will not changes 
        width_offset = int((width - height)/2)  ## Centers the new width
        width = height                          ## Turns area into a square
    crop_height = int(height/divide)    ##Divides image height into thirds
    crop_width = int(width/divide)      ##Divides image width into thirds
    crop_image = img[crop_height + height_offset + height_exception:height-crop_height + height_offset + height_exception, 
                     crop_width+ width_offset + width_exception:width-crop_width + width_offset + width_exception] ##1/3 in height, 1/3 in width.
    
    cv2.imwrite(new_dir + plot + "_" + year + month + day + ".tif", crop_image)  ##Plot#_yyyymmdd.tif
  
    



        

## Running the function

The final cell will crop the images and save them to a new directory. First, the new directory will be made. Then, the tool crop_image will be created. The tool first takes the date of the directory and adds a leading "0" if the month or day are single digits. It then opens the image and uses the shape to divide it into thirds. Finally, it will create a cropped image and save it to a new directory with a new name. 

For most of the images, moving the center 500 pixels up and 500 pixels to the right will avoid the cart bar. For the images on September 29th, the cart bar was still seen on the left side of the image after the initial crop, so an additional 500 pixels were added to the width to add it further to the right. Finally, plot 209B on October 25th had sensing equipment in the upper right, so the image was moved to the upper left.  

A final note is that images with "labels" in them will be skipped.

In [4]:

for folder in glob.glob(src_dir + "*"):
    date= folder.split("\\")[-1]              ##Windows only
    print("Working on images in folder " + date + "...\n")
    month, day, year = date.split(".")
    if date == "9.29.2021":
        for jpeg in glob.glob(folder + "\*.jpg"):
            image = jpeg.split("\\")[-1] #plot.jpg        Windows only
            
            if "label" in image:         ##Skipping images with labels
                pass
            else:
                crop_image(jpeg, image, year, month, day, 3, height_exception = -500, width_exception = 1000)
    else:
        for jpeg in glob.glob(folder + "\*.jpg"):
            image = jpeg.split("\\")[-1] #plot.jpg
            if "label" in image:         ##Skipping images with labels
                pass
            elif date == "10.25.2021" and image  =="209B.jpg":
                crop_image(jpeg, image, year, month, day, 3, height_exception = -500, width_exception = -700)
            else:
                crop_image(jpeg, image, year, month, day, 3, -500, 500)
    print("Done with date " + date + "...\n")
print("Cropping Images Complete!!!")


Working on images in folder 10.25.2021...

Done with date 10.25.2021...

Working on images in folder 7.8.2021...

Done with date 7.8.2021...

Working on images in folder 8.21.2021...

Done with date 8.21.2021...

Working on images in folder 10.15.2021...

Done with date 10.15.2021...

Working on images in folder 9.9.2021...

Done with date 9.9.2021...

Working on images in folder 9.29.2021...

Done with date 9.29.2021...

Working on images in folder 10.05.2021...

Done with date 10.05.2021...

Cropping Images Complete!!!


This notebook is complete. The next notebook will use these images to create a dataframe and collect the average green value from each image. 